##코로나 관련 트윗 분석
1. 2020년 5월14일-16일 동안 생성된 '코로나19' 키워드를 가진 트윗 10,000개 수집(크롤링)
2. 전처리 (불용어 제거)
3. FastText로 워드임베딩 생성 (skipgram, minCount=10)
4. Nearest neigbors (관련어) 출력
5. 벡터공간 시각화

In [0]:
# GetOldTweet3 라이브러리로 트윗 크롤링
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
#1. 트윗 크롤링
#2020년5월14-16일간 생성된 '코로나19' 키워드를 가진 트윗을 크롤링
#맥스트윗은 10,000개로 제한. HTTP 429 “Too Many Requests” 에러 방지
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('코로나19')\
                                           .setSince("2020-05-14")\
                                           .setUntil("2020-05-16")\
                                           .setMaxTweets(10000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

tweets=[]
for cnt, tw in enumerate(tweet):
  tweets.append(tw.text)
  if cnt%100==0:
    print(tw.text)
  
print("total %d tweet compiled" %len(tweets)

학교교육은 해방이후 미군정기시절 만든 체제를 거의 유지시켜왔는데 이번에 코로나19같은 '재난'을 당해서야 드디어 새로운 상상을 펼칠 수 있게된것 같습니다.9월학기제도 오래전에 얘기됐던건데 요즘 교육계에서 다시 부각되고 있습니다. 마침 오늘 학교교육의 새로운 방향에 대한 토론회가 있네요. 
[신천지 교리비교] 좌골신경통, 무릎관절은 물러가라! https://cafe.naver.com/miniatureheaven/6808 #신천지교리비교 #신천지예수교회 #성체성사 #성찬식 #유월절 #코로나19
#국민의당 "코로나19로 개학 연기…#수능 2회로 확대해야" 출처 : 뉴시스 | 네이버 http://naver.me/GNKAXLKm
신천지예수교회 ’코로나19’ 입장발표에 즈음하여 https://cafe.naver.com/miniatureheaven/6672 #신천지 #코로나19 #가짜뉴스
렘데시비르 긴급 승인 미 FDA 코로나19 치료제 
코로나19 31번째 확진자, 그리고 이만희 총회장님에 대하여 http://naver.me/FlWQzpmV #코로나19 #31번확진자 #이만희총회장 #신천지 https://twitter.com/zooroo616/status/1236300430629433347/photo/1
[홍성구의 뉴스브리핑] 2020년 5월 15일(금) - CDC 재개방 의사결정 도구 발표 "학교? 안돼!" - SK이노베이션, 조지아주 코로나 대응에 40만불 기부 - 뷰티마스터, 조지아주 재난청에 마스크 10만장 기부 - 아마존 물류센터서 6번째 코로나19 사망자 나와 
[속보] 성북구청, 코로나19 고양시 42번 확진자 관내 이동동선 공개…안암동 쭌이네 곱돌이, 이스포츠Pc아레나 http://www.topstarnews.net/news/articleView.html?idxno=783606 #mustread #feedly
그러게 촉세가 코로나19통걸렷나 !!
이번 코로나19로 인해서 주변의 스낵들이 꽤나 문을 닫는 모습을 보면서 언제 그만둬야하나 고민 하시던 분들이 가

In [0]:
#df=pd.read_csv("output_got.csv")
#tweet.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus.csv', encoding='utf-8-sig')
#크롤링한 GetOldTweets3 모델 타입을 처리가능한 리스트로 변환
total=[]
for tw in tweet:
  total.append(tw.text)
print(total[0])

학교교육은 해방이후 미군정기시절 만든 체제를 거의 유지시켜왔는데 이번에 코로나19같은 '재난'을 당해서야 드디어 새로운 상상을 펼칠 수 있게된것 같습니다.9월학기제도 오래전에 얘기됐던건데 요즘 교육계에서 다시 부각되고 있습니다. 마침 오늘 학교교육의 새로운 방향에 대한 토론회가 있네요. 


In [0]:
#원본 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus.csv', encoding='utf-8-sig', index=False)

In [0]:
#2. 한국어 전처리 (불용어제거) - 코엔엘파이 라이브러리 사용
!pip3 install konlpy    # Python 3.x

In [0]:
#불용어 제거
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
pprint(kkma.nouns(total[3]))
for i in range(len(total)):
  total[i]=" ".join(kkma.nouns(total[i]))

print(total[3])

['마스크',
 '코로나',
 '코로나19',
 '19',
 '안전',
 '안전벨트',
 '벨트',
 '이태원',
 '방문',
 '거짓',
 '거짓진술',
 '진술',
 '강사',
 '감염',
 '학생',
 '교회',
 '곳',
 '등',
 '방역',
 '방역수칙',
 '수칙',
 '추가',
 '추가감염',
 '복',
 '복교회',
 '사랑',
 '사랑장로교회',
 '장로',
 '교인',
 '여러분',
 '인천',
 '진자',
 '0',
 '뉴스']
마스크 코로나 코로나19 19 안전 안전벨트 벨트 이태원 방문 거짓 거짓진술 진술 강사 감염 학생 교회 곳 등 방역 방역수칙 수칙 추가 추가감염 복 복교회 사랑 사랑장로교회 장로 교인 여러분 인천 진자 0 뉴스


In [0]:
#전처리된 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus_pp.csv', encoding='utf-8-sig', index=False)

In [0]:
#3. Fasttext로 워드임베딩 생성 (model='skipgram', minCount=10)
!pip install fasttext
import fasttext

# Skipgram model :
model = fasttext.train_unsupervised('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus_pp.csv', model='skipgram', minCount=15)

# or, cbow model :
#model = fasttext.train_unsupervised('', model='cbow')

In [0]:
print(model.words)   # list of words in dictionary
print(model['코로나19']) # get the vector of the word '코로나19'

['</s>', '코로나', '19', '코로나19', '뉴스', '5', '일', '1', '명', '출처', '신천지', '진자', '시', '수', '2', '네이버', '이태원', '3', '월', '감염', '진', '확산', '관련', '클럽', '등', '다음', '방역', '교회', '발생', '중', '검사', '기부', '가짜', '만', '4', '극복', '기', '전', '6', '대응', '가짜뉴스', '신종', '체크', '14', '서울', '데', '한국', '대', '우리', '팩트', '구', '미국', '팩트체크', '지원', '지역', '15', '바', '마스크', '분', '5월', '사람', '발', '경제', '부', '바이러스', '연예', '0', '중국', '8', '사태', '해외', '내', '년', '예수', '김', '환자', '9', '연합', '전파', '예방', '현황', '사회', '방문', '팬', '이', '정부', '거리', '차', '신문', '속', '긴급', '신천지예수교회', '위', '세계', '이러스', '치료', '나', '경기', '집단', '생활', '코로나바이러스', '위기', '센터', '이후', '원', '20', '상황', '의료', '의료진', '2020', '기자', '14일', '번', '재난', '7', '다음뉴스', '감사', '10', '개', '배', '병원', '추가', '대구', '감염증', '방', '도', '안전', '오늘', '국민', '정국', '신규', '인천', '선', '안', '격리', '건강', '확인', '거', '로그', '미', '최대', '지원금', '온라인', '국내', '매일', '대책', '기준', '렘', '15일', '일본', '렘데시비르', '이유', '연기', '본부', '판정', '영향력', '베이스', '팬베이스', '백신', '매일신문', '선한', '30', '집', '때', '정', '가능', '말', '동'

In [0]:
model.get_nearest_neighbors('미국')

[(0.8780220746994019, '트럼프'),
 (0.8690294027328491, '한국산'),
 (0.8485060930252075, '미국선'),
 (0.8286957144737244, '급증'),
 (0.8215884566307068, '니'),
 (0.8206176161766052, '비데'),
 (0.8199236989021301, '제품'),
 (0.8091574311256409, '중국선'),
 (0.801011860370636, '프랑스'),
 (0.80010586977005, '뉴욕')]

In [0]:
!python3 -m pip install -U pygame==2.0.0.dev6 --user
#!apt-get install python-pygame
!pip install git+https://github.com/e9t/PyTagCloud.git

     |████████████████████████████████| 15.2MB 311kB/s 
  Found existing installation: pygame 1.9.6
    Uninstalling pygame-1.9.6:
      Successfully uninstalled pygame-1.9.6
  Cloning https://github.com/e9t/PyTagCloud.git to /tmp/pip-req-build-caielhrf
  Running command git clone -q https://github.com/e9t/PyTagCloud.git /tmp/pip-req-build-caielhrf
  Created wheel for pytagcloud: filename=pytagcloud-0.3.5-cp36-none-any.whl size=16467702 sha256=66393dfdc0bb41fa9bc82675196c1af7eb09665b9352b39ab9823bc04eacae44
  Stored in directory: /tmp/pip-ephem-wheel-cache-ijkbr3zq/wheels/7c/dd/2a/1b943312df4faec266e7e02a767ecbc246f3cd19d37c45e4c6
Successfully built pytagcloud


In [0]:
# create a virtualenv called 'anenv' and use it.
!python3 -m virtualenv anenv
!./anenv/bin/activate
# venvdotapp helps the python be a mac 'app'. So the pygame window can get focus.
!python -m pip install venvdotapp
!venvdotapp
!python -m pip install pygame

# See if pygame works with the oo module, and the aliens example.
!python -m pygame.examples.aliens

/usr/bin/python3: No module named virtualenv
/bin/bash: ./anenv/bin/activate: No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/venvdotapp", line 8, in <module>
    sys.exit(_main())
  File "/usr/local/lib/python3.6/dist-packages/venvdotapp.py", line 179, in _main
    require_bundle()
  File "/usr/local/lib/python3.6/dist-packages/venvdotapp.py", line 163, in require_bundle
    exe = _prepare_bundle(bundle_id)
  File "/usr/local/lib/python3.6/dist-packages/venvdotapp.py", line 142, in _prepare_bundle
    "Sorry, your Python wasn't built with --enable-framework."
venvdotapp.NotAFrameworkPython: Sorry, your Python wasn't built with --enable-framework.
pygame 2.0.0.dev6 (SDL 2.0.10, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc

In [0]:
#! /usr/bin/python2.7
# -*- coding: utf-8 -*-
from collections import Counter
import urllib
import random
import webbrowser

from konlpy.tag import Hannanum
from lxml import html
import pytagcloud # requires Korean font support
import sys

if sys.version_info[0] >= 3:
    urlopen = urllib.request.urlopen
else:
    urlopen = urllib.urlopen


r = lambda: random.randint(0,255)
color = lambda: (r(), r(), r())

def get_corona_text(cor_text):
    url = 'http://pokr.kr/bill/%s/text' % cor_text
    response = urlopen(url).read().decode('utf-8')
    page = html.fromstring(response)
    text = page.xpath(".//div[@id='bill-sections']/pre/text()")[0]
    return text

def get_tags(text, ntags=50, multiplier=10):
    h = Hannanum()
    nouns = h.nouns(text)
    count = Counter(nouns)
    return [{ 'color': color(), 'tag': n, 'size': c*multiplier }\
                for n, c in count.most_common(ntags)]

def draw_cloud(tags, filename, fontname='Noto Sans CJK', size=(800, 600)):
    pytagcloud.create_tag_image(tags, filename, fontname=fontname, size=size)
    webbrowser.open(filename)


cor_text = '한국'
text = get_corona_text(cor_text)
tags = get_tags(text)
print(tags)
draw_cloud(tags, 'wordcloud.png')

ModuleNotFoundError: ignored

In [0]:
model.get_word_id('한국')

47

In [0]:
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.words:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [0]:
tsne_plot(model)

NameError: ignored